In [2]:
# load both benign documents 
import json
from tqdm import tqdm
import csv

In [3]:
# load embedding model library
from google import genai
from google.genai import types
# load qwen 0.6b embedding
from transformers import AutoTokenizer, AutoModel, AutoModelForCausalLM 
import torch


/u/anp407/.cache/conda_envs/unknownattack/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [ ]:
# load qwen 0.6B embedding 
e_tokenizer = AutoTokenizer.from_pretrained("/u/anp407/Workspace/Huggingface/Qwen/Qwen3-Embedding-0.6B", trust_remote_code=True)
e_model = AutoModel.from_pretrained("/u/anp407/Workspace/Huggingface/Qwen/Qwen3-Embedding-0.6B", trust_remote_code=True, device_map="auto", dtype=torch.float16)
e_model.train() # we need the gradients
# use gemma 3 4b it to generate the answer
g_tokenizer = AutoTokenizer.from_pretrained("/u/anp407/Workspace/Huggingface/Qwen/Qwen3-4B-Instruct-2507", trust_remote_code=True)
g_model = AutoModelForCausalLM.from_pretrained("/u/anp407/Workspace/Huggingface/Qwen/Qwen3-4B-Instruct-2507", trust_remote_code=True, device_map="auto", dtype=torch.float16)
g_model.eval() # inference only
print("loaded models, qwen embedding and Qwen3 4b")

Loading checkpoint shards: 100%|██████████| 3/3 [00:01<00:00,  1.93it/s]

loaded models, qwen embedding and gemma 3 4b


In [5]:
adv_documents = json.load(open("nq_gemini_enriched.json", "r"))

In [6]:
# random a golden trigger
import random
# position is begin, mid and end
positions = ["begin", "mid", "end"]
num_tokens = [5, 10, 15]
target_gradient_guidance = True
device = 'cuda:0'
p_steps = 10
# p_target = ['query', 'document']
p_target = ['query']
steer = '-1'

queries = []
for key in adv_documents:
    queries.append(adv_documents[key]['question'])
benign_docs = []
for key in adv_documents:
    benign_docs.append(adv_documents[key]['benign_responses'])
adv_docs = []
for key in adv_documents:
    adv_docs.append(adv_documents[key]['adv_texts'])

In [ ]:
for pos in positions:
    for num_token in num_tokens:
        # we tried different positions and num_tokens, here we just use one example
        # we have only one adv passage, but 10 benign passages, we want to add a trigger both to the adv passages and query to increase its similarity
        
        # if the p_target is query, we only add trigger to the query
        if 'query' in p_target:
            for i, query in tqdm(enumerate(queries)):
                trigger_ids = [e_tokenizer.pad_token_id] * num_token
                trigger_token_list = e_tokenizer.convert_ids_to_tokens(trigger_ids)
                trigger_ids = torch.tensor(trigger_ids, device='cuda:0').unsqueeze(0)
                trigger_attention_mask = torch.ones_like(trigger_ids, device='cuda:0')
                
                query_ids = e_tokenizer(query, add_special_tokens=False, return_tensors="pt").to("cuda:0")
                if pos == "begin":
                    p_query_ids = torch.cat([trigger_ids, query_ids['input_ids']], dim=1)
                    p_query_attention_mask = torch.cat([trigger_attention_mask, query_ids['attention_mask']], dim=1)
                    p_query_ids_fixed = 
                elif pos == "mid":
                    mid_index = query_ids['input_ids'].size(1) // 2
                    p_query_ids = torch.cat([query_ids['input_ids'][:, :mid_index], trigger_ids, query_ids['input_ids'][:, mid_index:]], dim=1)
                    p_query_attention_mask = torch.cat([query_ids['attention_mask'][:, :mid_index], trigger_attention_mask, query_ids['attention_mask'][:, mid_index:]], dim=1)
                elif pos == "end":
                    p_query_ids = torch.cat([query_ids['input_ids'], trigger_ids], dim=1)
                    p_query_attention_mask = torch.cat([query_ids['attention_mask'], trigger_attention_mask], dim=1)
                    
                # get the query_embeddings
                p_query_embeddings = e_model(input_ids=p_query_ids, attention_mask=p_query_attention_mask)
                
                # get the document embeddings
                b_embeddings = []
                for doc in benign_docs[i]:
                    doc_ids = e_tokenizer(doc, add_special_tokens=False, return_tensors="pt").to("cuda:0")
                    doc_embeddings = e_model(input_ids=doc_ids['input_ids'], attention_mask=doc_ids['attention_mask'])
                    b_embeddings.append(doc_embeddings)
                a_doc_ids = e_tokenizer(adv_docs[i][0], add_special_tokens=False, return_tensors="pt").to("cuda:0")
                a_doc_embeddings = e_model(input_ids=a_doc_ids['input_ids'], attention_mask=a_doc_ids['attention_mask'])
            
                # iterate p_steps to optimize the trigger
                q_embedding_before = e_model.get_input_embeddings()(p_query_ids)
                q_embedding_after = e
                 
                for step in range(p_steps):
                    # compute similarity loss
                    b_min_similarities = []
                    for b_emb in b_embeddings:
                        similarity = torch.cosine_similarity(p_query_embeddings.last_hidden_state[:,-1,:], b_emb.last_hidden_state[:,-1,:])
                        b_min_similarities.append(similarity)
                    b_min_sim = torch.min(torch.stack(b_min_similarities))
                    
                    a_similarity = torch.cosine_similarity(p_query_embeddings.last_hidden_state[:,-1,:], a_doc_embeddings.last_hidden_state[:,-1,:])
                    # compute loss
                    loss = - a_similarity + b_min_sim
                    # backpropagate
                    loss.backward()
                    # get the gradient
                    
                    
                    

0it [00:00, ?it/s]/tmp/ipykernel_2715574/1722610131.py:55: UserWarning: The .grad attribute of a Tensor that is not a leaf Tensor is being accessed. Its .grad attribute won't be populated during autograd.backward(). If you indeed want the .grad field to be populated for a non-leaf Tensor, use .retain_grad() on the non-leaf Tensor. If you access the non-leaf Tensor by mistake, make sure you access the leaf Tensor instead. See github.com/pytorch/pytorch/pull/30531 for more information. (Triggered internally at /pytorch/build/aten/src/ATen/core/TensorBody.h:489.)
  trigger_embeds -= trigger_embeds.grad * 0.1
0it [00:00, ?it/s]


TypeError: unsupported operand type(s) for *: 'NoneType' and 'float'

In [ ]:
# Running optimization with GCG
ids_suffix_best = ids_suffix.clone()
best_loss = float("inf")
all_losses = []
for step in tqdm(range(steps), desc="Optimization steps", unit="step"):
    # Getting input embeds of current suffix
    one_hot = torch.nn.functional.one_hot(ids_suffix, num_classes=model.config.vocab_size).to(model.device, model.dtype)
    one_hot.requires_grad = True
    embeds_suffix = one_hot @ model.get_input_embeddings().weight

    # Getting gradients w.r.t one-hot encodings
    cache_copy = deepcopy(kv_cache) # In recent versions of huggingface's transformers, we need a copy of the cache to avoid getting gradients multiple times w.r.t the same tensors
    loss = model(
        inputs_embeds=torch.cat([embeds_suffix, embeds_after], dim=1),
        labels=labels,
        past_key_values=cache_copy,
        use_cache=True
    ).loss
    loss.backward()
    gradients = -one_hot.grad
    
    # Updating best suffix ever
    all_losses.append(loss.item())
    if loss.item() < best_loss:
        best_loss = loss.item()
        ids_suffix_best = ids_suffix.clone()

    # Getting top-k tokens for all positions (candidate substitutions)
    top_k_tokens = torch.topk(gradients, top_k, dim=-1).indices

    # Creating a batch with substitutions and storing losses
    sub_positions = torch.randint(0, suffix_length, (batch_size,))
    sub_tokens = torch.randint(0, top_k, (batch_size,))
    batch = ids_suffix.clone().repeat(batch_size, 1)
    for idx, (position, token) in enumerate(zip(sub_positions, sub_tokens)):
        batch[idx, position] = top_k_tokens[0, position, token]

    # Computing losses for the batch (in sub mini-batches)
    losses = []
    for slice_start in range(0, batch_size, search_batch_size):
        slice_end = min(slice_start + search_batch_size, batch_size)
        ids_slice = batch[slice_start: slice_end]
        
        with torch.no_grad():
            # Getting loss for the batch
            try:
                batch_kv_cache_copy = deepcopy(batch_kv_cache)
                logits = model(
                    input_ids=torch.cat([ids_slice, ids_after.repeat(ids_slice.shape[0], 1)], dim=1),
                    past_key_values=batch_kv_cache_copy,
                    use_cache=True
                ).logits[:, -ids_after.shape[1]: -1]

                # Getting losses
                losses.extend([
                    torch.nn.functional.cross_entropy(logits[i], ids_after[0, 1:]).item()
                    for i in range(search_batch_size)
                ])
            except Exception as e:
                print(f"Exception: {e}")
                print("Exception during forward pass. If OOM, try reducing the search batch size.")
                break

    # Updating the suffix
    best_idx = np.argmin(losses)
    best_position, best_token = sub_positions[best_idx].item(), sub_tokens[best_idx].item()
    ids_suffix[0, best_position] = top_k_tokens[0, best_position, best_token]

    # Logging infos
    mean_loss = np.mean(losses)
    print(f"Step {step + 1}/{steps} | Best loss: {best_loss:.3f} | Current loss: {loss.item():.3f} | Mean loss: {mean_loss}")
    visualize(ids_before, ids_suffix, ids_after)

In [10]:
suffix_placeholder = '|SUFFIX_PLACEHOLDER|'
messages = [
    {"role": "system", "content": "system_prompt"},
    {"role": "user", "content": "request" + '|SUFFIX_PLACEHOLDER|'},
    {"role": "assistant", "content": "target"},
]

# Getting text before and after the suffix
text = e_tokenizer.apply_chat_template(messages, add_generation_prompt=False, tokenize=False)
text_before, text_after = text.split(suffix_placeholder)

In [12]:
print(text_before )
print('------')
print(text_after)

<|im_start|>system
system_prompt<|im_end|>
<|im_start|>user
request
------
<|im_end|>
<|im_start|>assistant
<think>

</think>

target<|im_end|>



In [ ]:
trigger_embeds.grad

tensor([[[-0.0043,  0.0435, -0.0334,  ..., -0.0039,  0.0449,  0.0320],
         [-0.0043,  0.0435, -0.0334,  ..., -0.0039,  0.0449,  0.0320],
         [-0.0043,  0.0435, -0.0334,  ..., -0.0039,  0.0449,  0.0320],
         [-0.0043,  0.0435, -0.0334,  ..., -0.0039,  0.0449,  0.0320],
         [-0.0043,  0.0435, -0.0334,  ..., -0.0039,  0.0449,  0.0320]]],
       device='cuda:1', dtype=torch.float16)

In [ ]:
p_query_embeddings.hidden_states

BaseModelOutputWithPast(last_hidden_state=tensor([[[  3.3613,  -7.2148,  -0.1650,  ...,  -8.4219, -12.9375,   0.3081],
         [  3.3613,  -7.2148,  -0.1650,  ...,  -8.4219, -12.9375,   0.3081],
         [  3.3613,  -7.2148,  -0.1650,  ...,  -8.4219, -12.9375,   0.3081],
         ...,
         [ -1.7715,  -1.2158,  -0.2764,  ...,   2.0938,  -3.4062,  -0.3687],
         [ -1.4912,  -1.7412,  -0.6958,  ...,   1.3066,  -2.0957,  -1.4893],
         [ -3.2285,  -5.7148,  -0.4097,  ...,  -1.7334,  -7.3594,  -1.2129]]],
       device='cuda:0', dtype=torch.float16, grad_fn=<ToCopyBackward0>), past_key_values=DynamicCache(layers=[DynamicLayer, DynamicLayer, DynamicLayer, DynamicLayer, DynamicLayer, DynamicLayer, DynamicLayer, DynamicLayer, DynamicLayer, DynamicLayer, DynamicLayer, DynamicLayer, DynamicLayer, DynamicLayer, DynamicLayer, DynamicLayer, DynamicLayer, DynamicLayer, DynamicLayer, DynamicLayer, DynamicLayer, DynamicLayer, DynamicLayer, DynamicLayer, DynamicLayer, DynamicLayer, Dynami